In [1]:
import numpy as np
import pandas as pd
import sys
import getpass
import re
from functions import *
from collections import Counter
import copy
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')

ImportError: No module named pymarkovchain

In [ ]:
df = pd.read_csv('C:\Users\\'+getpass.getuser()+'\\Google Drive\Sarah Ido folder\data\CCK\MATCHING_phet_cck_user_actions+sophistication_WITHPAUSE_anonymized.txt')
df_scores = pd.read_csv('C:\Users\\'+getpass.getuser()+'\\Google Drive\Sarah Ido folder\data\CCK\MATCHING_phet_cck_user_data_anonymized.txt')
df["student"] = df["student"].astype('category')
df_scores["student"] = df_scores["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()

Let's get the sequences by students in blocks of actions.

## Here is how we get sequences 

In [ ]:
# df2 = df[df.Activity == 'a1']
# blocks = get_blocks(df2,get_students())
# frequencies = get_frequencies(blocks, shortest=3, longest=10)
# #Count number of time sequences occurs accross all students
# sum_frequencies = sum(frequencies.values(), Counter())
# #Count number of students that conducted that frequency
# counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})

## Here is how to select certain sequences

In [ ]:
# ## Clean up sequences with count less than N
# N = 5
# remove_rare_frequencies(counts_frequencies,N)
# # keep_frequencies_with(counts_frequencies,keep='P')
# frequencies = clean_student_frequencies(counts_frequencies, frequencies)
# # counts_frequencies.most_common(10)

## Let's compare sequences from students in the different conditions in Activity 1
Get all frequencies of length 3 to 10, cleaning them up by removing those common to all students and the rare ones.

In [ ]:
# summed_seqs, seq_scaff, seq_not = mega_process(activity='a1', level='scaffolding', value1='scaff', value2='not', shortest=3, longest=10)
# difference(seq_scaff,seq_not,N=10)

We can now look at the difference in number of students that conduct sequences in both groups.

In [ ]:
## Does this difference persist in activity 2?

In [ ]:
# summed_seqs, seq_scaff, seq_not = mega_process(activity='a2', level='scaffolding', value1='scaff', value2='not', shortest=3, longest=10)
# difference(seq_scaff,seq_not,N=10)

## Let's compare sequences from students from different clusters
In activity 1

In [ ]:
# summed_seqs, seq_know, seq_att = mega_process(activity='a1', level='Clustergroups', value1='knowledge', value2='attitude', shortest=3, longest=10)
# difference(seq_know+seq_know,seq_att,N=10)

**Note**: Here we are doubling the number of counts for the knowledge group since it's half the size

In activity 2

In [ ]:
# summed_seqs, seq_know, seq_att = mega_process(activity='a2', level='Clustergroups', value1='knowledge', value2='attitude', shortest=3, longest=10)
# difference(seq_know+seq_know,seq_att,N=10)

**Note**: Here we are doubling the number of counts for the knowledge group since it's half the size

## Let's compare sequences from students with high and low learning gain

First we need to split students by learning gains in each activity

In [ ]:
def label_learning (row,column):
    if row[column] >= median_learning: return 'high'
    else: return 'low'

df_scores['learning1score'] = df_scores["post t1"] - df_scores["pre"]
median_learning = np.median(df_scores['learning1score'])
df_scores['learning1'] = df_scores.apply (lambda row: label_learning (row,"learning1score"),axis=1)

df_scores['learning2score'] = df_scores["post t2"] - df_scores["pre"]
median_learning = np.median(df_scores['learning2score'])
df_scores['learning2'] = df_scores.apply (lambda row: label_learning (row,"learning2score"),axis=1)
# len(df_scores[(df_scores['learning1']=='high') & (df_scores['learning2']=='high')])

Now we look at the different in sequences in activity 1

In [ ]:
# summed_seqs, seq_high, seq_low = mega_process(activity='a1', level='learning1', value1='high', value2='low', shortest=3, longest=10)
# difference(seq_high,seq_low,N=10)



And activity 2

In [ ]:
# summed_seqs, seq_high, seq_low = mega_process(activity='a2', level='learning2', value1='high', value2='low', shortest=3, longest=10)
# difference(seq_high,seq_low,N=10)

## Well, one thing we are really interested in, is what happens before/after students pause. So let's look at those sequences, and compare low/high learners.

In [ ]:
# summed_seqs, seq_high, seq_low = mega_process(activity='a1', level='learning1', value1='high', 
#                                               value2='low', shortest=3, longest=10, keep = "P")
# difference(seq_high,seq_low,N=10)

In [ ]:
# summed_seqs, seq_high, seq_low = mega_process(activity='a2', level='learning2', value1='high', 
#                                               value2='low', shortest=3, longest=10, keep = "P")
# difference(seq_high,seq_low,N=10)

#### Now let's pick a sequence visualize how much it's used by students

In [ ]:
def plot_seq_gains(activity, seqs):
    df2 = df[df.Activity == activity]
    if activity == 'a1': score_column = "learning1score"
    else: score_column = "learning2score"
        
    blocks = get_blocks(df2,get_students())
    frequencies = get_frequencies(blocks, shortest=3, longest=6)
    sum_frequencies = sum(frequencies.values(), Counter())
    counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})
    totals = {student:float(sum(count.values())) for student,count in frequencies.iteritems()}

    fig, axs = plt.subplots(nrows=len(seqs),ncols=1,figsize=(15,3*len(seqs)))
    fig.suptitle('Normalize frequency of usage of sequences versus learning gain. Students are colored depending on their scaffolding.',fontsize=16)
    for seq,ax in zip(seqs,axs):
        counts = keep_frequencies_with(counts_frequencies,keep=seq)
        cleaned_frequencies = clean_student_frequencies(counts, frequencies)

        f1 = select_frequencies(cleaned_frequencies,"scaffolding","scaff")
        f2 = select_frequencies(cleaned_frequencies,"scaffolding","not")

        total1 = {student:float(sum(count.values()))/totals[student] for student,count in f1.iteritems()}
        values1 = {k:(v, float(df_scores[df_scores["student"]==k][score_column]) ) for k,v in total1.iteritems()}
        total2 = {student:float(sum(count.values()))/totals[student] for student,count in f2.iteritems()}
        values2 = {k:(v, float(df_scores[df_scores["student"]==k][score_column]) ) for k,v in total2.iteritems()}
        x,y = zip(*values1.values())
        rec1 = ax.scatter(x,y, c='r',s=100, alpha=0.4)
        x,y = zip(*values2.values())
        rec2 = ax.scatter(x,y, c='b',s=100, alpha=0.4)
        ax.axhline(0, color='darkgrey')
        ax.set_ylabel(seq)
        ax.legend((rec1, rec2), ('Scaffolded', 'Not scaffolded'))
        
#         ax.set_xlabel('Normalized frequency of usage')
#         ax.set_ylabel('Learning gain in activity 1 (post-pre)')
#         ax.set_title('Normalize frequency of usage of {0} versus learning gain. Students are colored depending on their scaffolding.'.format(seq))

### Want to make a matrix where rows are sequences, columns students, cells represent number of times sequence was performed by student

In [ ]:
df2 = df[df.Activity == 'a1']
blocks = get_blocks(df2,get_students())
frequencies = get_frequencies(blocks, shortest=3, longest=6)
sum_frequencies = sum(frequencies.values(), Counter())
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})
totals = {student:float(sum(count.values())) for student,count in frequencies.iteritems()}

students, sequences = frequencies.keys(), sum_frequencies.keys()
A = np.zeros((len(students), len(sequences)))
A.shape

for st,seqs in frequencies.iteritems():
    i = students.index(st)
    for sq,count in seqs.iteritems():
        j = sequences.index(sq)
        A[i][j] = count

We normalize the matrix (in reality the right way to normalize is to divide by number of blocks of sequences per student. Right now, the number of sequences per students depends on how diverse their set of seqs is)

In [ ]:
blocks

In [ ]:
B = np.zeros((len(students), len(sequences)))
for i,row in enumerate(A):
    B[i,:] = A[i,:]/len(blocks[students[i]])

We calculate correlations with learning outcomes for each sequence

In [ ]:
gains = [round(float(df_scores[df_scores["student"]==student]['learning1score']),5) for student in students]

from scipy.stats import pearsonr
correlation_measure = pearsonr
corr = []
for sq in sequences:
    j = sequences.index(sq)
    if np.count_nonzero(B[:,j]) > 10:
        c = correlation_measure(B[:,j],gains)
        corr.append((sq, c[0], c[1]))    

Now let's keep only the significant sequences.

**Note**: we still need to do some boostrapping and permutation test to confirm that these are significant results.

In [ ]:
sig_corr = []
for s,c,p in corr:
    p = p/2.0
    if p <= 0.05 and abs(c) > 0.25:
        sig_corr.append((s,c,p))
        print s,c,p        

In [ ]:
# plot_seq_gains('a1',zip(*sig_corr)[0])

Clearly, how much students did an action doesn't matter as much as who did it at least once (or more?)
Let's repeat the analysis using # of students who did the sequence instead of frequency.

In [ ]:
df2 = df[df.Activity == 'a1']
blocks = get_blocks(df2,get_students())
frequencies = get_frequencies(blocks, shortest=3, longest=6)
sum_frequencies = sum(frequencies.values(), Counter())
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})
totals = {student:float(sum(count.values())) for student,count in frequencies.iteritems()}

students, sequences = frequencies.keys(), sum_frequencies.keys()
A = np.zeros((len(students), len(sequences)))
A.shape


for st,seqs in frequencies.iteritems():
    i = students.index(st)
    for sq,count in seqs.iteritems():
        j = sequences.index(sq)
        if count >= 1: x =1
        else: x = 0
        A[i][j] = x

In [ ]:
from scipy.stats import ttest_ind
gains = np.array([round(float(df_scores[df_scores["student"]==student]['learning1score']),5) for student in students])

tests = []
seq_gains = {}
for sq in sequences:
    j = sequences.index(sq)
    a = gains[np.where(A[:,j]==1)] #learning gains of student who used sequence
    b = gains[np.where(A[:,j]==0)] #learning gains of student who didn't
    t = ttest_ind(a, b, equal_var=False)
    tests.append((sq, t[0], t[1]))
    seq_gains[sq] = (a,b)

In [ ]:
sig_tests = []
print "Sequence, # students/{0}, t-test, p-value".format(len(students))
for s,t,p in tests:
    p = p/2.0
    if p <= 0.05:
        n = len(seq_gains[s][0]) #number of students who did the sequence
        if n > 10 and n < 86: # and 'P' in s:
            sig_tests.append((s,n,t,p))

# order by ttest value
results = sorted(sig_tests, key=lambda x: x[2])
for s,n,t,p in results:
    print "{0} \t\t {1}\t {2} \t {3}".format(s, n, round(t,2), round(p,3))

In [ ]:
plt.figure()
seq = 'ROPT'
means = [np.mean(seq_gains[seq][0]),np.mean(seq_gains[seq][1])]
stdes = [np.std(seq_gains[seq][0]),np.std(seq_gains[seq][1])]
plt.boxplot(seq_gains[seq])
plt.ylabel('Average learning gain')
plt.xticks(range(1,3),[str(len(seq_gains[seq][0])) +' did it', str(len(seq_gains[seq][1])) + ' didn\'t'])
plt.title('Learning gains depending on use of sequence '+seq)
plt.grid()